# Stokes equation
We consider a Stokes problem on a channel with a parabolic inflow on the left and an obstacle placed inside. On the right we have an outflow boundary (do-nothing Neumann boundary condition). At the top and bottom wall and the obstacle we prescribe so-called no-slip boundary conditions $u=0$.

Find $(u,p)\in V\times Q$ such that
\begin{align*}
& -\Delta u -\nabla p = 0 &&\text{ in } \Omega \\
& \text{div}(u) = 0 &&\text{ in } \Omega \\
& u = u_{in} &&\text{ on } \Gamma_{\text{inlet}}\\
& u = 0 &&\text{ on } \Gamma_{\text{wall}}\cup \Gamma_{\text{cyl}}
\end{align*}

In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.geom2d import SplineGeometry

In [2]:
geo = SplineGeometry()
geo.AddRectangle( (0, 0), (2, 0.41), bcs = ("wall", "outlet", "wall", "inlet"))
geo.AddCircle( (0.2, 0.2), r=0.05, leftdomain=0, rightdomain=1, bc="cyl")
mesh = Mesh( geo.GenerateMesh(maxh=0.05))

uin = CoefficientFunction( (1.5*4*y*(0.41-y)/(0.41*0.41),0) )
Draw(Norm(uin), mesh, "inlet")#,deformation=True)

NGSWebGuiWidget(value={'ngsolve_version': '6.2.2008', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': F…

## Crouzeix-Raviart-$P^0(\mathcal T)$ Stokes element

In [3]:
V = FESpace("nonconforming", mesh, order=1, dirichlet="wall|inlet|cyl")
Q = H1(mesh, order = 0)

X = V*V*Q

In [4]:
ux,uy,p = X.TrialFunction()
vx,vy,q = X.TestFunction()

div_u = grad(ux)[0]+grad(uy)[1]
div_v = grad(vx)[0]+grad(vy)[1]

a = BilinearForm(X)
a += (grad(ux)*grad(vx)+grad(uy)*grad(vy) + div_u*q + div_v*p) * dx
a.Assemble()

In [5]:
f = LinearForm(X)
f.Assemble()

In [6]:
gfu = GridFunction(X)
uin_1 = 1.5*4*y*(0.41-y)/(0.41*0.41)
gfu.components[0].Set(uin_1, definedon=mesh.Boundaries("inlet"))

velocity = CoefficientFunction(gfu.components[0:2])
pressure = CoefficientFunction(gfu.components[2])

In [7]:
gfu.components[0].Set(uin_1, definedon=mesh.Boundaries("inlet"))
#c = Preconditioner(a,"local")   #<- Jacobi preconditioner
c = Preconditioner(a,"direct") #<- sparse direct solver
c.Update()
solvers.BVP(bf=a, lf=f, gf=gfu, pre=c)
Redraw()

iteration 0 error = 8.494482708461543
iteration 1 error = 2.924308927234865e-14


In [8]:
Draw(velocity, mesh, "vel")
Draw(pressure, mesh, "pressure")
SetVisualization(max=2)

NGSWebGuiWidget(value={'ngsolve_version': '6.2.2008', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': F…

NGSWebGuiWidget(value={'ngsolve_version': '6.2.2008', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': F…

## Taylor-Hood elements

In [9]:
V = VectorH1(mesh, order = 2, dirichlet = "wall|inlet|cyl")
Q = H1(mesh, order = 0)

In [10]:
u = V.TrialFunction()
p = Q.TrialFunction()
v = V.TestFunction()
q = Q.TestFunction()


a = BilinearForm(V)
a += InnerProduct(grad(u),grad(v)) * dx

b = BilinearForm(trialspace=V, testspace=Q)
b += div(u)*q*dx

a.Assemble()
b.Assemble()

In [11]:
mp = BilinearForm(Q)
mp += p*q*dx
mp.Assemble()

In [12]:
f = LinearForm(V)
f.Assemble()

g = LinearForm(Q)
g.Assemble()

In [13]:
gfu = GridFunction(V, name="u")
gfp = GridFunction(Q, name="p")

gfu.Set(uin, definedon=mesh.Boundaries("inlet"))

In [14]:
gfu.Set(uin, definedon=mesh.Boundaries("inlet"))
K = BlockMatrix( [ [a.mat, b.mat.T], [b.mat, None] ] )
C = BlockMatrix( [ [a.mat.Inverse(V.FreeDofs()), None], [None, mp.mat.Inverse()] ] )

rhs = BlockVector ( [f.vec, g.vec] )
sol = BlockVector( [gfu.vec, gfp.vec] )

solvers.MinRes (mat=K, pre=C, rhs=rhs, sol=sol, initialize=False)

it =  0  err =  4.226349524222936
it =  1  err =  2.24742720237691
it =  2  err =  2.0079965248753786
it =  3  err =  1.5223974591861806
it =  4  err =  1.4865165128148892
it =  5  err =  1.2684172936012543
it =  6  err =  1.2238227706232239
it =  7  err =  1.0613991811698644
it =  8  err =  0.9751048985880616
it =  9  err =  0.8059667668187446
it =  10  err =  0.7659685044672495
it =  11  err =  0.6732666269471119
it =  12  err =  0.6441496546283989
it =  13  err =  0.5885055784970951
it =  14  err =  0.5761085504203545
it =  15  err =  0.5350751953698764
it =  16  err =  0.5284519959422413
it =  17  err =  0.4992634648147116
it =  18  err =  0.4790130012830835
it =  19  err =  0.4501314537196627
it =  20  err =  0.4383073011697616
it =  21  err =  0.4009282353966521
it =  22  err =  0.35857850978473527
it =  23  err =  0.30279166557592146
it =  24  err =  0.28138958292648175
it =  25  err =  0.17696934093687483
it =  26  err =  0.16827009411896102
it =  27  err =  0.11721391516867231

basevector

In [15]:
Draw(gfu)
Draw(gfp)

NGSWebGuiWidget(value={'ngsolve_version': '6.2.2008', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': F…

NGSWebGuiWidget(value={'ngsolve_version': '6.2.2008', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': F…